In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('remap_(764)_[1619]_20221031_13_29_55_0730_dvoice_progression_add_dod_demrv_date-AD.csv')

In [3]:
df["date_"] = df["date"].apply(lambda s:datetime(year=int(str(s)[0:4]), month=int(str(s)[4:6]), day=int(str(s)[6:8])))
df["date_of_death_"] = df["date_of_death"].apply(lambda s:datetime(year=int(str(s)[0:4]), month=int(str(s)[4:6]), day=int(str(s)[6:8])) if pd.notnull(s) else datetime(year=2021, month=6, day=15))
df["review_date_str_"] = df["review_date_str"].apply(lambda s:datetime(year=int(str(s)[0:4]), month=int(str(s)[4:6]), day=int(str(s)[6:8])) if pd.notnull(s) else s)

In [4]:
df["delta_death"] = df["date_of_death_"] - df["date_"]
df["delta_review"] = df["review_date_str_"] - df["date_"]
df["delta_death"] = df["delta_death"].apply(lambda x:x.days if pd.notnull(x) else +10000)
df["delta_review"] = df["delta_review"].apply(lambda x:x.days)

In [5]:
df['Patientid'] = (df['idtype']).astype(str)+'-'+(df['id']).astype(str)

In [6]:
df_norm = df[df.is_norm == 1]
df_mci = df[df.is_mci == 1]
df_demented = df[df.is_demented == 1]

In [7]:
def transition(data,days,type_T):
    assert type_T in ("NtoMCI","MCItoD")
    if type_T == "NtoMCI":
        df_norm = data.copy()
        a_nn = ((df_norm.days_to_mci.isna()) & 
        (df_norm.days_to_mild_dementia.isna()) &
        (df_norm.days_to_moderate_dementia.isna()) & 
        (df_norm.days_to_severe_dementia.isna()) & 
        ((df_norm.delta_death>days) & (df_norm.delta_review>days)))
        
        b_nn = ((df_norm.days_to_mci>days) | ((df_norm.days_to_mci.isna()) &
        (df_norm.days_to_mild_dementia>days)) | ((df_norm.days_to_mci.isna()) &
        (df_norm.days_to_mild_dementia.isna()) & (df_norm.days_to_severe_dementia>days)))
        
        df_NN = df_norm[a_nn|b_nn]
        print(f"number of unique N to N {df_NN.id_date.nunique()} in {days/365} years")
        
        a_nmci = ((df_norm.days_to_mci<days) | ((df_norm.days_to_mci.isna()) & (df_norm.days_to_mild_dementia<days)) | 
                  ((df_norm.days_to_mci.isna()) & (df_norm.days_to_mild_dementia.isna()) & 
                   (df_norm.days_to_moderate_dementia<days)))
        df_NMCI = df_norm[a_nmci]
        print(f"number of unique N to MCI {df_NMCI.id_date.nunique()} in {days/365} years")
        return df_NN, df_NMCI
    else:
        df_mci = data.copy()
        a_mcimci = ((df_mci.days_to_mild_dementia.isna()) & (df_mci.days_to_moderate_dementia.isna()) & 
                    (df_mci.days_to_severe_dementia.isna()) &((df_mci.delta_death>days) & (df_mci.delta_review>days)))
        b_mcimci = ((df_mci.days_to_mild_dementia>days) | ((df_mci.days_to_mild_dementia.isna()) &
                    (df_mci.days_to_moderate_dementia>days)) | ((df_mci.days_to_mild_dementia.isna()) &
                    (df_mci.days_to_moderate_dementia.isna()) & (df_mci.days_to_severe_dementia>days)))
        df_MCIMCI = df_mci[a_mcimci | b_mcimci]
        print(f"number of unique MCI to MCI {df_MCIMCI.id_date.nunique()} in {days/365} years")
        
        a_mcid = ((df_mci.days_to_mild_dementia<days)|((df_mci.days_to_mild_dementia.isna()) &
                    (df_mci.days_to_moderate_dementia<days)) |((df_mci.days_to_mild_dementia.isna()) &
                    (df_mci.days_to_moderate_dementia.isna()) & (df_mci.days_to_severe_dementia<days)))
        df_MCID = df_mci[a_mcid]
        print(f"number of unique MCI to D {df_MCID.id_date.nunique()} in {days/365} years")
        return df_MCIMCI,df_MCID
        

In [ ]:
# for n_year in [1,2,3,4,5,6,7]:
#     df1,df2 = transition(df_norm,365*n_year,"NtoMCI")
#     df3,df4 = transition(df_mci,365*n_year,"MCItoD")
#     df1.to_pickle(f"data/NtoN_{n_year}.pkl")
#     df2.to_pickle(f"data/NtoMCI_{n_year}.pkl")
#     df3.to_pickle(f"data/MCItoMCI_{n_year}.pkl")
#     df4.to_pickle(f"data/MCItoD_{n_year}.pkl")